## start session

In [15]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('').getOrCreate()

## read data

In [20]:
training=spark.read.csv('/home/ubuntu/CryptoBot/regression/SalaryData.csv',  header=True, inferSchema=True, nullValue="NULL")

In [21]:
training.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
| Sunny| 52|        10| 30000|
|   May| 19|         5| 20000|
|   Bob| 35|         4| 40000|
|  Paul| 42|         2| 15000|
|Mahesh| 36|         4| 50000|
| Masha| 35|         5| 20000|
+------+---+----------+------+



In [22]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [23]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [25]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")


In [26]:
output=featureassembler.transform(training)

In [27]:
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
| Sunny| 52|        10| 30000|         [52.0,10.0]|
|   May| 19|         5| 20000|          [19.0,5.0]|
|   Bob| 35|         4| 40000|          [35.0,4.0]|
|  Paul| 42|         2| 15000|          [42.0,2.0]|
|Mahesh| 36|         4| 50000|          [36.0,4.0]|
| Masha| 35|         5| 20000|          [35.0,5.0]|
+------+---+----------+------+--------------------+



In [28]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [29]:
finalized_data=output.select("Independent Features","Salary")

In [30]:
from pyspark.ml.regression import LinearRegression
##traintest split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features",labelCol="Salary")
regressor=regressor.fit(train_data)


25/01/28 13:57:12 WARN Instrumentation: [d76307d3] regParam is zero, which might cause numerical instability and overfitting.
25/01/28 13:57:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/01/28 13:57:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [31]:
### Coefficents
regressor.coefficients


DenseVector([1185.1852, -5768.7991])